In [139]:
from enum import Enum
import matplotlib.pyplot as plt
import plotly.graph_objects as go

In [140]:
class Label(Enum):
    UNKNOWN = 1
    EMPTY = 2
    OCCUPIED = 3
    VIEWED = 4
    RANGE = 5
    CAMERA = 6

In [141]:
class Cell:
    def __init__(self, x, y, z, label):
        self.x = x
        self.y = y
        self.z = z
        self.label = label
    
    def set_label(self, label):
        self.label = label
        return self
    
    def __str__(self):
        return f"(({self.x},{self.y},{self.z}), {self.label})"
    
    def __repr__(self):
        return f"(({self.x},{self.y},{self.z}), {self.label})"

In [142]:
class BoundingBox:
    def __init__(self, bounding_box_list):
        self._bbox = bounding_box_list
    def get_bounding_box(self):
        return self._bbox
    def contains_cell(self, cell):
        for (bbox_coord_lower, bbox_coord_upper), cell_coord in zip(self._bbox, [cell.x, cell.y, cell.z]):
            if cell_coord < bbox_coord_lower or cell_coord > bbox_coord_upper:
                return False
        return True
    def get_middle_point(self):
        return (self._bbox[0][0] + self._bbox[0][1])/2,  (self._bbox[1][0] + self._bbox[1][1])/2, (self._bbox[2][0] + self._bbox[2][1])/2
    def intersects(self, other):
        return not (
            other._bbox[0][0] > self._bbox[0][1] or  
            other._bbox[0][1] < self._bbox[0][0] or
            other._bbox[1][0] > self._bbox[1][1] or
            other._bbox[1][1] < self._bbox[1][0] or
            other._bbox[2][0] > self._bbox[2][1] or
            other._bbox[2][1] < self._bbox[2][0]
        )
    def draw(self, fig):

        fig.add_trace(go.Mesh3d(
            # 8 vertices of a cube
            x=[self._bbox[0][0], self._bbox[0][0], self._bbox[0][1], self._bbox[0][1], self._bbox[0][0], self._bbox[0][0], self._bbox[0][1], self._bbox[0][1]],
            y=[self._bbox[1][0], self._bbox[1][1], self._bbox[1][1], self._bbox[1][0], self._bbox[1][0], self._bbox[1][1], self._bbox[1][1], self._bbox[1][0]],
            z=[self._bbox[2][0], self._bbox[2][0], self._bbox[2][0], self._bbox[2][0], self._bbox[2][1], self._bbox[2][1], self._bbox[2][1], self._bbox[2][1]],
            i = [7, 0, 0, 0, 4, 4, 6, 6, 4, 0, 3, 2],
            j = [3, 4, 1, 2, 5, 6, 5, 2, 0, 1, 6, 3],
            k = [0, 7, 2, 3, 6, 7, 1, 1, 5, 5, 7, 6],
            opacity=0.6,
            color='#DC143C',
            flatshading = True

        ))





In [143]:
class Octree:
    def __init__(self, cell):
        # leaf
        self._cells = [cell]
    def __init__(self, boundary: BoundingBox, max_nodes = 8, depth = 0):
        self._max_nodes = max_nodes
        self._cells = []
        self._depth = depth
        self._boundary: BoundingBox = boundary
        self._divided = False
        self._children = [None]*8
    
    def set_cells(self, cells):
        self._cells = cells
        return self
    
    def set_depth(self, depth):
        self.depth = depth
        return self
    
    def copy(self):
        return Octree(self._max_nodes, self._boundary, self._max_nodes).set_cells(self._cells).set_depth(self._depth)

    def find_cells(self, boundary: BoundingBox, found_cells):
        if not self._boundary.intersects(boundary):
            return False
        
        for cell in self._cells:
            if boundary.contains_cell(cell):
                found_cells.append(cell)
        if self._divided:
            for child in self._children:
                child.find_cells(boundary, found_cells)
        return found_cells

    def insert(self, cell):
        if not self._boundary.contains_cell(cell):
            return False
        if len(self._cells) < self._max_nodes and self._children[0] is None:
            self._cells.append(cell)
            return True
        if not self._divided:
            self.divide()
        
        for child in self._children:
            if (child.insert(cell)):
                return True
        
        return False
    
    def get_all_cells(self, found_cells):
        for cell in self._cells:
            found_cells.append(cell)
        if self._divided:
            for child in self._children:
                child.get_all_cells(found_cells)
        return found_cells
        
        
    def divide(self):
        mid_x, mid_y, mid_z = self._boundary.get_middle_point()
        bbox = self._boundary.get_bounding_box()
        
        self._children[0] = Octree(BoundingBox(
            [(bbox[0][0], mid_x),
            (bbox[1][0], mid_y),
            (bbox[2][0], mid_z)]
            ),
            depth=self._depth + 1
        )
        self._children[1] = Octree(BoundingBox(
            [(mid_x, bbox[0][1]),
            (bbox[1][0], mid_y),
            (bbox[2][0], mid_z)]
            ),
            depth=self._depth + 1
        )
        self._children[2] = Octree(BoundingBox(
            [(mid_x, bbox[0][1]),
            (mid_y, bbox[1][1]),
            (bbox[2][0], mid_z)]
            ),
            depth=self._depth + 1
        )
        self._children[3] = Octree(BoundingBox(
            [(bbox[0][0], mid_x),
            (mid_y, bbox[1][1]),
            (bbox[2][0], mid_z)]
            ),
            depth=self._depth + 1
        )
        self._children[4] = Octree(BoundingBox(
            [(bbox[0][0], mid_x),
            (bbox[1][0], mid_y),
            (mid_z, bbox[2][1])]
            ),
            depth=self._depth + 1
        )
        self._children[5] = Octree(BoundingBox(
            [(mid_x, bbox[0][1]),
            (bbox[1][0], mid_y),
            (mid_z, bbox[2][1])]
            ),
            depth=self._depth + 1
        )
        self._children[6] = Octree(BoundingBox(
            [(mid_x, bbox[0][1]),
            (mid_y, bbox[1][1]),
            (mid_z, bbox[2][1])]
            ),
            depth=self._depth + 1
        )
        self._children[7] = Octree(BoundingBox(
            [(bbox[0][0], mid_x),
            (mid_y, bbox[1][1]),
            (mid_z, bbox[2][1])]
            ),
            depth=self._depth + 1
        )

        for cell in self._cells:
            for child in self._children:
                if child.insert(cell):
                    break
        self._cells = []

        self._divided = True
    def __len__(self):
        length = len(self._cells)
        if self._divided:
            length += sum(len(child) for child in self._children)
        return length
    def draw(self, fig):
        self._boundary.draw(fig)
        if self._divided:
            for child in self._children:
                child.draw(fig)
                
        
        

In [144]:
class UnderwaterMap:
    def update_map(self):
        pass
    def compute_detections(self):
        pass


In [145]:
cells = []
with open("point_cloud.xyz", "r") as file:
    for line in file:
        cells.append(Cell(*[float(x) for x in line.split()], Label.CAMERA))

In [146]:
ot = Octree(BoundingBox([(-5,35),(-30,30), (-40, -20)]))

In [147]:
i = 0
for cell in cells:
    if i > 100:
        break
    i += 1
    res = ot.insert(cell)
    if not res:
        print(cell.x, cell.y, cell.z)


In [148]:
len(ot.get_all_cells([]))

101

In [149]:
len(ot)

101

In [150]:
fig = go.FigureWidget()
ot.draw(fig)

In [174]:
fig

FigureWidget({
    'data': [{'color': '#DC143C',
              'flatshading': True,
              'i': [7, 0, …